In [14]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser

In [15]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [16]:
RDS_HOST = 'tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com'
print(RDS_HOST)

tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com


In [17]:
postgres_driver = f"""postgresql://{config.get('TRANSACC', 'DB_USER')}:{config.get('TRANSACC', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('TRANSACC', 'DB_PORT')}/{config.get('TRANSACC', 'DB_NAME')}"""

#### Creamos Dimensión de Personas


In [18]:
sql_query = 'SELECT * FROM persona;'
df_personas = pd.read_sql(sql_query, postgres_driver)
df_personas.head()

,idpersona,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email
0,1536244591,empleado,Dillon Frost,Pasaporte,162945912451763,"440 Megan Manors\nSarahstad, SC 64628",38089217,ashley36@hotmail.com
1,1268532496,cliente,Matthew Richards,Pasaporte,155290750418394,"627 Jean Hill Suite 778\nWest Tanyaview, WI 52390",36755108,josephacevedo@hotmail.com
2,1968500916,empleado,Derek Cruz,DNI,193686991975379,"055 Cabrera Points Suite 960\nCindyhaven, WV 4...",36555616,aguirresandra@yahoo.com
3,1109002691,cliente,Ms. Susan Freeman,DNI,158855813307991,"71873 Massey Garden Apt. 052\nSouth Sarah, MO ...",35166718,bbaker@hotmail.com
4,1085777313,proveedor,Bethany Woods,Pasaporte,127352387081821,"91283 Richard Meadow\nEast Tonya, MH 15417",34275114,krystalbarber@hotmail.com


In [19]:
sql_query = 'SELECT * FROM ingreso;'
df_ingreso = pd.read_sql(sql_query, postgres_driver)
df_ingreso.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado
0,0,1735433711,5674321,Nota de Crédito,855596,None,2010-06-26,37.16,55055.88,Completada
1,1,1173940292,1361098,Recibo,938348,None,2010-06-26,76.82,84670.31,Cancelada
2,2,1469297972,5417720,Boleta de Venta,561283,None,2010-06-26,28.48,14607.33,Completada
3,3,1516543853,4719555,Factura,232494,None,2010-06-26,90.64,75219.66,Completada
4,4,1654902231,1729649,Nota de Débito,139725,None,2010-06-26,87.98,89359.46,Completada


In [20]:
df_ingreso = df_ingreso.rename(columns={'idproveedor': 'idpersona'})

In [21]:
dimPersonas = df_personas.merge(df_ingreso, how='inner', on='idpersona')
dimPersonas.drop(['idingreso', 'idusuario'], axis=1,  inplace=True)
dimPersonas.head()

,idpersona,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado
0,1536244591,empleado,Dillon Frost,Pasaporte,162945912451763,"440 Megan Manors\nSarahstad, SC 64628",38089217,ashley36@hotmail.com,Ticket de Venta,897809,None,2010-06-26,98.83,6448.96,Cancelada
1,1109002691,cliente,Ms. Susan Freeman,DNI,158855813307991,"71873 Massey Garden Apt. 052\nSouth Sarah, MO ...",35166718,bbaker@hotmail.com,Nota de Crédito,568942,None,2010-06-26,93.41,70995.04,Completada
2,1109002691,cliente,Ms. Susan Freeman,DNI,158855813307991,"71873 Massey Garden Apt. 052\nSouth Sarah, MO ...",35166718,bbaker@hotmail.com,Recibo,204749,None,2010-06-26,65.68,1792.07,Completada
3,1109002691,cliente,Ms. Susan Freeman,DNI,158855813307991,"71873 Massey Garden Apt. 052\nSouth Sarah, MO ...",35166718,bbaker@hotmail.com,Factura,104596,None,2010-06-26,81.81,17411.70,Cancelada
4,1926261339,proveedor,Brooke Chang,Pasaporte,185501290907297,"74501 Johnson Ramp\nNathanview, TX 62418",38326477,daniellelane@hotmail.com,Nota de Débito,110259,None,2010-06-26,39.05,67167.20,Completada


#### Dimensión de Usuario
